In [1]:
import os
os.environ['KERAS_BACKEND'] = 'torch'  # or 'tensorflow' or 'jax'

In [2]:
import torch
import keras
import optuna
import numpy as np
import mlflow
import mlflow.keras

from mlflow.keras import MlflowCallback
from mlflow.models import infer_signature, ModelSignature
from mlflow.types import Schema, TensorSpec

In [3]:
# The set_experiment API creates a new experiment if it doesn't exist.
mlflow.set_tracking_uri("http://127.0.0.1:5000")                # Point to the remote MLflow server via REST API
mlflow.set_experiment("MNIST with Keras within MLFLow")         # Set the experiment name

# IMPORTANT: Enable system metrics monitoring
mlflow.config.enable_system_metrics_logging()                   # Enable system metrics logging
mlflow.config.set_system_metrics_sampling_interval(1)           # The frequency of logging of the metrics. By default it's 10s, but here we set it to 1s
mlflow.config.set_system_metrics_samples_before_logging(3)      # The number of samples to aggregate

# The actual logging time window is the product of MLFLOW_SYSTEM_METRICS_SAMPLING_INTERVAL and MLFLOW_SYSTEM_METRICS_SAMPLES_BEFORE_LOGGING. For example, if the sample interval is set to 2 seconds and samples before logging to 3, then system metrics will be collected every 2 seconds, then after 3 samples are collected (2 * 3 = 6s), we aggregate the metrics and log to MLflow server.

In [4]:
# Load and prepare data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [5]:
device = None

if torch.cuda.is_available():
    device = torch.device("cuda")
    device_name = torch.cuda.get_device_name(0)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Alternative using accelerate library if PyTorch version >= 2.6
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [6]:
def create_model():
    model = keras.Sequential(
        [
            keras.Input(shape=(784,)),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(10, activation='softmax')
        ]
    )

    return model

In [7]:
# Training parameters
params = {"epochs": 10,
          "batch_size": 128,
          "learning_rate": 0.001,
          "optimizer": "adam",
          "loss_function": "categorical_crossentropy",
          "dropout_rate": 0.2,
          "hidden_units": [512, 256]}

In [8]:
# Create and compile model
model = create_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=params["learning_rate"]),
              loss=params["loss_function"],
              metrics=["accuracy"])

In [9]:
# Enable autologging for Pytorch/Keras
mlflow.keras.autolog(registered_model_name="Keras_MNIST_autolog_model", # Name of the model in MLflow Model Registry
                     log_models=True,                                   # Save the model in MLflow Model Registry at the end of the run
                     log_every_n_steps=params['batch_size'],            # Log metrics every n steps (batches)  
                     log_every_epoch=False,                             # Disabling logging metrics at the end of every epoch as we are logging every n steps
                     log_model_signatures=True,                         # Log the model signature (input and output schema)  
)

In [10]:
with mlflow.start_run(run_name='autolog_run') as run:
    if os.environ['KERAS_BACKEND'] == 'torch':
        if device.type == 'cuda' or device.type == 'mps':
            print("Training Pytorch model on GPU/TPU...\n")
            
            x_train_torch = torch.tensor(x_train).to(device)    # Creating a torch tensor version of x_train
            x_test_torch = torch.tensor(x_test).to(device)      # Creating a torch tensor version of x_test
            
            model.fit(x_train_torch,
                        y_train,
                        batch_size=params["batch_size"],
                        epochs=params["epochs"],
                        validation_data=(x_test_torch, y_test),
                        verbose=1)
        else:
            print("GPU/TPU not available. Training Pytorch model on CPU...\n")
            model.fit(x_train,
                    y_train,
                    batch_size=params["batch_size"],
                    epochs=params["epochs"],
                    validation_data=(x_test, y_test),
                    verbose=1)
        

2025/12/12 12:24:43 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:24:43 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Pytorch model on GPU/TPU...



2025/12/12 12:24:53 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'torch.Tensor'>. Dataset logging skipped.
2025/12/12 12:24:53 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: int() argument must be a string, a bytes-like object or a real number, not 'builtin_function_or_method'


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9189 - loss: 0.2697 - val_accuracy: 0.9635 - val_loss: 0.1168
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9672 - loss: 0.1081 - val_accuracy: 0.9749 - val_loss: 0.0753
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9768 - loss: 0.0766 - val_accuracy: 0.9778 - val_loss: 0.0707
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9805 - loss: 0.0597 - val_accuracy: 0.9804 - val_loss: 0.0622
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9840 - loss: 0.0487 - val_accuracy: 0.9813 - val_loss: 0.0588
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9869 - loss: 0.0400 - val_accuracy: 0.9818 - val_loss: 0.0655
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9883 - loss: 0.0345 - val_accuracy: 0.9817 - val_loss: 0.0600
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9893 - loss: 0.0317 - val_accu

2025/12/12 12:25:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'Keras_MNIST_autolog_model' already exists. Creating a new version of this model...
2025/12/12 12:25:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Keras_MNIST_autolog_model, version 11
Created version '11' of model 'Keras_MNIST_autolog_model'.
2025/12/12 12:26:00 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 12:26:00 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run autolog_run at: http://127.0.0.1:5000/#/experiments/7/runs/7cc365da7d144aa283b7477b9655471c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7


In [11]:
# Get the last run (autologging created it)
last_run = mlflow.last_active_run()   # or mlflow.search_runs() to query

# Re‑attach
with mlflow.start_run(run_id=last_run.info.run_id):

    # Evaluate model
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    mlflow.log_metrics({"test_loss": test_loss, "test_accuracy": test_accuracy})

print(f"Test accuracy: {test_accuracy:.4f}")

mlflow.end_run()

2025/12/12 12:26:00 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:26:00 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/12/12 12:26:02 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 12:26:02 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run autolog_run at: http://127.0.0.1:5000/#/experiments/7/runs/7cc365da7d144aa283b7477b9655471c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7
Test accuracy: 0.9834


In [12]:
# Stopping autolog runs so that it doesn't interfere with future runs
mlflow.autolog(disable=True)
mlflow.keras.autolog(disable=True)

# Manually Logging Keras Experiments

While MLflow provides seamless autologging integration with Keras/TensorFlow via either **mlflow.tensorflow.autolog()** or **mlflow.autolog()**, it works only with **model.fit()** and needs to be executed before we invoke model training via the same. In comparison, if we wish to customise our model training, we need to manually log all the details we wish to capture.

For more control over what gets logged, you can manually instrument your Keras training code using MLflow's logging APIs:

**mlflow.log_metric() / mlflow.log_metrics()**: Log metrics such as accuracy and loss during training
**mlflow.log_param() / mlflow.log_params()**: Log parameters such as learning rate and batch size
**mlflow.keras.log_model()**: Save your Keras model to MLflow
**mlflow.log_artifact()**: Log artifacts such as model checkpoints and plots

## Using MLflow's Keras Callback

MLflow provides a built-in callback for Keras that simplifies experiment tracking. The **mlflow.keras.MlflowCallback()** integrates seamlessly with your Keras training loop. The MlflowCallback supports various configuration options. Some of these options are also available out-of-box for **mlflow.autolog()**. We have demonstrated some of the options in the previous code.

In [13]:
# Use MLflow's built-in callback
mlflow_callback = MlflowCallback(log_every_epoch=False, log_every_n_steps=params['batch_size'])       # Log metrics every 5 batches instead of every epoch

with mlflow.start_run(run_name="manual_run",
                      log_system_metrics=True) as run:

    if os.environ['KERAS_BACKEND'] == 'torch':
        if device.type == 'cuda' or device.type == 'mps':
            print("Training Pytorch model on GPU/TPU...\n")

            x_train_torch = torch.tensor(x_train).to(device)    # Creating a torch tensor version of x_train
            x_test_torch = torch.tensor(x_test).to(device)      # Creating a torch tensor version of x_test
            
            history = model.fit(x_train_torch,
                                y_train,
                                batch_size=params["batch_size"],
                                epochs=params["epochs"],
                                validation_data=(x_test_torch, y_test),
                                callbacks=[mlflow_callback],
                                verbose=1)
        else:
            print("GPU/TPU not available. Training Pytorch model on CPU...\n")
            history = model.fit(x_train,
                                y_train,
                                batch_size=params["batch_size"],
                                epochs=params["epochs"],
                                validation_data=(x_test, y_test),
                                callbacks=[mlflow_callback],
                                verbose=1)
            
            
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    mlflow.log_metrics({"test_loss": test_loss, "test_accuracy": test_accuracy})

    # Prepare sample data for signature inference
    sample_input = x_test[:100]
    sample_predictions = model.predict(sample_input)
    # Infer signature from sample data
    signature = infer_signature(sample_input, sample_predictions)

    # Define input and output schemas, in case infer_signature is not working as desired
    # input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 784))])
    # output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 10))])
    # signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log the trained model
    mlflow.keras.log_model(model, name="Keras_MNIST_manual_model", signature=signature)

    print(f"Test accuracy: {test_accuracy:.4f}")


2025/12/12 12:26:02 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:26:02 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Pytorch model on GPU/TPU...



Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9913 - loss: 0.0253 - val_accuracy: 0.9827 - val_loss: 0.0618
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9925 - loss: 0.0222 - val_accuracy: 0.9831 - val_loss: 0.0655
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9924 - loss: 0.0225 - val_accuracy: 0.9844 - val_loss: 0.0643
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9932 - loss: 0.0204 - val_accuracy: 0.9835 - val_loss: 0.0675
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9937 - loss: 0.0196 - val_accuracy: 0.9835 - val_loss: 0.0700
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9948 - loss: 0.0156 - val_accuracy: 0.9838 - val_loss: 0.0705
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9949 - loss: 0.0146 - val_accuracy: 0.9830 - val_loss: 0.0762
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9940 - loss: 0.0176 - val_accu

2025/12/12 12:27:02 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 12:27:02 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run manual_run at: http://127.0.0.1:5000/#/experiments/7/runs/4b46cf11ef10408e935382a4c2ed6a42
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7


In [14]:
# Load and use the model
model_info = mlflow.keras.log_model(model, name="Keras_MNIST_autolog_model")
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

# Make predictions
predictions = loaded_model.predict(x_test[:5])
print("Predictions:", predictions)

2025/12/12 12:27:02 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


2025/12/12 12:27:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predictions: [[2.41250874e-14 2.84437214e-13 5.25287618e-14 1.00005206e-10
  2.50366913e-18 3.33660869e-17 2.65959531e-19 1.00000000e+00
  5.19692406e-16 1.46417156e-09]
 [7.95408119e-14 2.30431869e-08 1.00000000e+00 9.23568134e-14
  2.45602179e-25 9.01774900e-17 2.22895128e-18 3.74190611e-15
  9.81608351e-17 4.22657460e-28]
 [4.37706658e-16 1.00000000e+00 1.85822406e-15 5.50810901e-17
  9.65283825e-11 1.05247903e-13 7.40667239e-13 1.76222384e-10
  1.53264557e-09 9.37823698e-15]
 [1.00000000e+00 6.30552237e-17 7.36533265e-11 8.76725857e-15
  4.20296190e-14 1.66238125e-15 1.27962108e-09 1.64620346e-12
  6.37423111e-13 1.61446939e-10]
 [1.26752400e-10 1.48517847e-11 9.90439154e-12 2.46111299e-16
  9.99994755e-01 3.87896150e-13 3.57848679e-13 6.91232005e-09
  2.57117660e-10 5.18005982e-06]]


## Hyperparameter Tuning with Keras and MLflow
Combine Keras with hyperparameter tuning libraries while tracking everything in MLflow

In [15]:
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    hidden_units = trial.suggest_categorical("hidden_units", [64, 128, 256, 512])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)

    with mlflow.start_run(nested=True):
        # Log trial parameters
        mlflow.log_params({"learning_rate": lr,
                           "batch_size": batch_size,
                           "hidden_units": hidden_units,
                           "dropout_rate": dropout_rate
                           })

        # Create model with suggested parameters
        model = keras.Sequential([keras.Input(shape=(784,)),
                                  keras.layers.Dense(hidden_units, activation='relu'),
                                  keras.layers.Dropout(dropout_rate),
                                  keras.layers.Dense(256, activation='relu'),
                                  keras.layers.Dropout(dropout_rate),
                                  keras.layers.Dense(10, activation='softmax')
                            ])

        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                      loss="categorical_crossentropy",
                      metrics=["accuracy"])
        
        if os.environ['KERAS_BACKEND'] == 'torch':
            if device.type == 'cuda' or device.type == 'mps':
                print("Training Pytorch model on GPU/TPU...\n")

                x_train_torch = torch.tensor(x_train).to(device)    # Creating a torch tensor version of x_train
                
                # Train model
                history = model.fit(x_train_torch,
                                    y_train,
                                    batch_size=batch_size,
                                    epochs=10,
                                    validation_split=0.2,
                                    verbose=1)
                
            else:
                print("GPU/TPU not available. Training Pytorch model on CPU...\n")
                history = model.fit(x_train,
                                    y_train,
                                    batch_size=batch_size,
                                    epochs=10,
                                    validation_split=0.2,
                                    verbose=1)
                
        # Evaluate model
        val_loss = max(history.history["val_loss"])
        val_accuracy = max(history.history["val_accuracy"])
        mlflow.log_metrics({"val_loss": val_loss, "val_accuracy": val_accuracy})

        return val_accuracy

In [16]:
# Run hyperparameter optimization
with mlflow.start_run():
    mlflow.set_tag("optimization", "optuna")
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    # Log best parameters
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_val_accuracy", study.best_value)

2025/12/12 12:27:09 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:27:09 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
[I 2025-12-12 12:27:09,263] A new study created in memory with name: no-name-6a57ae06-c4f5-4033-8427-b7894e278f9c
2025/12/12 12:27:09 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:27:09 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Pytorch model on GPU/TPU...

Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.4730 - loss: 1.7738 - val_accuracy: 0.7928 - val_loss: 1.0020
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.7554 - loss: 0.8675 - val_accuracy: 0.8693 - val_loss: 0.5244
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8174 - loss: 0.6134 - val_accuracy: 0.8937 - val_loss: 0.4043
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8460 - loss: 0.5162 - val_accuracy: 0.9026 - val_loss: 0.3513
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8659 - loss: 0.4571 - val_accuracy: 0.9097 - val_loss: 0.3183
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8754 - loss: 0.4192 - val_accuracy: 0.9163 - val_loss: 0.2952
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8856 - loss: 0.3891 - val_accuracy: 0.9206 - val_loss: 0.2768
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accu

2025/12/12 12:28:28 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 12:28:29 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[I 2025-12-12 12:28:29,002] Trial 0 finished with value: 0.9302499890327454 and parameters: {'learning_rate': 3.367454532946405e-05, 'batch_size': 64, 'hidden_units': 64, 'dropout_rate': 0.2046811041415484}. Best is trial 0 with value: 0.9302499890327454.
2025/12/12 12:28:29 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:28:29 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


🏃 View run mercurial-colt-426 at: http://127.0.0.1:5000/#/experiments/7/runs/8b307511ba584b0588f1e9ef0eef38b9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7
Training Pytorch model on GPU/TPU...

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.7558 - loss: 0.9671 - val_accuracy: 0.8988 - val_loss: 0.4272
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7744 - loss: 0.9113 - val_accuracy: 0.8899 - val_loss: 0.5097
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.7809 - loss: 0.9257 - val_accuracy: 0.8573 - val_loss: 0.7098
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.7694 - loss: 0.9832 - val_accuracy: 0.8798 - val_loss: 0.6021
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.7926 - loss: 0.9199 - val_accuracy: 0.8833 - val_loss: 0.5947
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.7915 - loss: 0.9518 - val_accuracy: 0.8941 - val_loss: 0.5025


2025/12/12 12:31:27 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 12:31:27 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[I 2025-12-12 12:31:27,281] Trial 1 finished with value: 0.8987500071525574 and parameters: {'learning_rate': 0.01228422893278514, 'batch_size': 32, 'hidden_units': 512, 'dropout_rate': 0.48629348360337743}. Best is trial 0 with value: 0.9302499890327454.
2025/12/12 12:31:27 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 12:31:27 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


...

🏃 View run unique-sponge-842 at: http://127.0.0.1:5000/#/experiments/7/runs/9c9a65f2b3fb428394dd36986cf89984
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7
Training Pytorch model on GPU/TPU...

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6988 - loss: 1.1399 - val_accuracy: 0.8914 - val_loss: 0.4364
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8756 - loss: 0.4413 - val_accuracy: 0.9166 - val_loss: 0.3050
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9010 - loss: 0.3443 - val_accuracy: 0.9283 - val_loss: 0.2580
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9148 - loss: 0.2978 - val_accuracy: 0.9350 - val_loss: 0.2302
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9245 - loss: 0.2623 - val_accuracy: 0.9417 - val_loss: 0.2075
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9321 - loss: 0.2355 - val_accuracy: 0.9463 - val_loss: 0.1900
Epoch 7/10
375/375 

2025/12/12 13:03:44 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 13:03:44 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[I 2025-12-12 13:03:44,600] Trial 18 finished with value: 0.9577500224113464 and parameters: {'learning_rate': 6.128977294150152e-05, 'batch_size': 128, 'hidden_units': 256, 'dropout_rate': 0.14803177532653766}. Best is trial 16 with value: 0.981166660785675.
2025/12/12 13:03:44 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/12/12 13:03:44 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


🏃 View run judicious-bear-417 at: http://127.0.0.1:5000/#/experiments/7/runs/90f349df97c2427988ee6ebd0a67d9fe
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7
Training Pytorch model on GPU/TPU...

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3461 - loss: 2.0060 - val_accuracy: 0.7293 - val_loss: 1.5024
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6760 - loss: 1.2925 - val_accuracy: 0.8287 - val_loss: 0.8596
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7680 - loss: 0.8711 - val_accuracy: 0.8658 - val_loss: 0.5837
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8115 - loss: 0.6760 - val_accuracy: 0.8848 - val_loss: 0.4642
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8351 - loss: 0.5732 - val_accuracy: 0.8964 - val_loss: 0.3973
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8524 - loss: 0.5100 - val_accuracy: 0.9028 - val_loss: 0.3578
Epoch 7/10
375/375

2025/12/12 13:04:23 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 13:04:23 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[I 2025-12-12 13:04:23,121] Trial 19 finished with value: 0.9198333621025085 and parameters: {'learning_rate': 1.1377478156038008e-05, 'batch_size': 128, 'hidden_units': 512, 'dropout_rate': 0.29212220301666797}. Best is trial 16 with value: 0.981166660785675.
2025/12/12 13:04:23 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 13:04:23 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run bouncy-midge-972 at: http://127.0.0.1:5000/#/experiments/7/runs/59d60d1f8dd44ce795fead058f6b8967
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7
🏃 View run aged-lark-122 at: http://127.0.0.1:5000/#/experiments/7/runs/e2b29108607b4bddae8414ba7d8cc4d5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7
